# Install all required dependencies

In [ ]:
!pip install pandas-ta tqdm

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from typing import Tuple
from tqdm import tqdm

# 1. Step 1: Works on the columns

Some columns are duplicated and redundant.

In [133]:
data_path = Path("btc_2023-2024_combined_data.csv")
df_raw = pd.read_csv(data_path)

In [209]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple

class OHLCVUnifier:
    """
    Unify OHLCV metrics from 'cg' and 'cq' sources into robust single series per metric.
    Specifically combines:
      - cg_c + cq_close → unified_close
      - cg_h + cq_high   → unified_high
      - cg_l + cq_low    → unified_low
      - cg_o + cq_open   → unified_open
    Uses MAD-based outlier filtering, then median aggregation, and drops original columns.
    """
    def __init__(self):
        # define the mapping of unified metric -> source column patterns
        self.mapping: Dict[str, List[str]] = {
            'close': ['cg_c', 'cq_close'],
            'high':  ['cg_h', 'cq_high'],
            'low':   ['cg_l', 'cq_low'],
            'open':  ['cg_o', 'cq_open'],
        }

    def unify(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
        """
        Returns:
          - df_out: DataFrame with new columns unified_<metric> and original cg_/cq_ cols removed
          - used_cols: dict mapping each metric to the list of columns that were combined
        """
        df_out = df.copy()
        used_cols: Dict[str, List[str]] = {}

        for metric, cols in self.mapping.items():
            # select only cols that actually exist
            present = [c for c in cols if c in df_out.columns]
            if not present:
                continue
            used_cols[metric] = present
            group = df_out[present]
            # median and MAD
            med = group.median(axis=1)
            mad = group.sub(med, axis=0).abs().median(axis=1)
            # mask out outliers beyond 3*MAD
            mask = group.sub(med, axis=0).abs().le(3 * mad, axis=0)
            filtered = group.where(mask)
            # unified result
            unified = filtered.median(axis=1)
            df_out[f'unified_{metric}'] = unified
            # drop original columns
            df_out.drop(columns=present, inplace=True)

        return df_out, used_cols

In [211]:
def detect_and_prepare_datetime(df: pd.DataFrame) -> pd.DataFrame:
    if "date" in df.columns:
        return df
    if "start_time" in df.columns:
        df["date"] = pd.to_datetime(df["start_time"], unit="ms")
        return df
    for col in df.columns:
        if col.endswith("_time"):
            df["date"] = pd.to_datetime(df[col], unit="s")
            return df
    raise ValueError("No suitable timestamp column found")

In [233]:
unifier = OHLCVUnifier()
df_unified, used = unifier.unify(df)
df_unified = detect_and_prepare_datetime(df_unified)

In [225]:
class ColumnAbbreviator:
    """
    Renames DataFrame columns by replacing full words with abbreviations.
    """
    # constant map: full word (singular/plural) -> abbreviation
    PART_ABBREV: dict[str, str] = {
        'transaction': 'txn',
        'transactions': 'txn',
        'address': 'addr',
        'addresses': 'addr',
        'blockchain': 'bc',
        'count': 'cnt',
        'counts': 'cnt',
        'exchange': 'exch',
        'exchanges': 'exch',
        'breakdown': 'brkdwn',
        'breakdowns': 'brkdwn',
        'liquidation': 'liq',
        'liquidations': 'liq',
    }

    @classmethod
    def abbreviate(cls, col: str) -> str:
        """
        Replace any segment in the column name that matches PART_ABBREV.
        """
        parts = col.split('_')
        return '_'.join(cls.PART_ABBREV.get(p, p) for p in parts)

    @classmethod
    def rename_df(cls, df: pd.DataFrame) -> pd.DataFrame:
        """
        Return a new DataFrame with its columns renamed via the abbreviate method.
        """
        return df.rename(columns=cls.abbreviate)

In [344]:
df_renamed = ColumnAbbreviator.rename_df(df_unified)

# Step 2: Feature Engineering

In [253]:
from dataclasses import dataclass
from typing import List, Callable, Dict
import pandas as pd
import numpy as np
import pandas_ta as ta

@dataclass
class BTCFeatureEngineer:
    """
    Clean, modular pipeline for BTC features: price returns, technical indicators, on-chain metrics.
    """
    key_date: str = 'date'
    price_col: str = 'unified_close'
    high_col: str = 'unified_high'
    low_col: str = 'unified_low'
    # core on-chain
    market_cap_col: str = 'market_cap'
    tx_volume_col: str = 'tx_volume_usd'
    realized_cap_col: str = 'realized_cap'
    active_addr_col: str = 'addresses_active_count'
    tx_count_col: str = 'transactions_count'
    cdd_col: str = 'coin_days_destroyed'
    hash_rate_col: str = 'hash_rate'
    difficulty_col: str = 'difficulty'
    mempool_txs_col: str = 'mempool_txs_count_sum'
    fear_greed_col: str = 'fear_greed_index'
    sopr_col: str = 'spent_output_profit_ratio'
    # additional on-chain (required by default)
    realized_price_col: str = 'realized_price'
    miner_outflows_col: str = 'miner_outflows'
    exchange_flow_col: str = 'exchange_flows'
    dormant_coins_col: str = 'avg_dormancy'
    hodl_waves_col: str = 'hodl_waves_1y_share'
    miner_revenue_col: str = 'miner_revenue'
    whale_balances_col: str = 'whale_balance_ratio'
    active_addrs_col: str = 'active_addresses'
    mev_col: str = 'mev_value'
    exchange_reserves_col: str = 'exchange_reserves'
    tx_fees_col: str = 'tx_fees_usd'

    def _validate(self, df: pd.DataFrame):
        missing = [c for c in [self.key_date, self.price_col, self.high_col, self.low_col]
                   if c not in df.columns]
        if missing:
            raise KeyError(f"Missing required columns: {missing}")

    def _set_index(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df[self.key_date] = pd.to_datetime(df[self.key_date])
        return df.set_index(self.key_date)

    def _clean(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.drop_duplicates().dropna()

    def _add_price_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df['ret']     = df[self.price_col].pct_change()
        df['log_ret'] = np.log(df[self.price_col] / df[self.price_col].shift(1))
        df['cum_ret'] = df['ret'].cumsum()
        return df

    def _add_moving_averages(self, df: pd.DataFrame, windows: List[int] = (7,21,50,100,200)) -> pd.DataFrame:
        for w in windows:
            df[f'sma_{w}'] = df[self.price_col].rolling(w).mean()
            df[f'ema_{w}'] = ta.ema(df[self.price_col], length=w)
        return df

    def _add_volatility(self, df: pd.DataFrame, window: int = 14) -> pd.DataFrame:
        df[f'vol_{window}'] = df['log_ret'].rolling(window).std()
        df['atr'] = ta.atr(df[self.high_col], df[self.low_col], df[self.price_col], length=window)
        bb = ta.bbands(df[self.price_col], length=window)
        df[['bb_up','bb_mid','bb_low']] = bb[[f'BBU_{window}_2.0', f'BBM_{window}_2.0', f'BBL_{window}_2.0']]
        return df

    def _add_momentum(self, df: pd.DataFrame) -> pd.DataFrame:
        df['rsi_14'] = ta.rsi(df[self.price_col], length=14)
        macd = ta.macd(df[self.price_col])
        df[['macd','macd_signal','macd_hist']] = macd[['MACD_12_26_9','MACDs_12_26_9','MACDh_12_26_9']]
        return df

    def _add_trend_strength(self, df: pd.DataFrame) -> pd.DataFrame:
        adx = ta.adx(df[self.high_col], df[self.low_col], df[self.price_col], length=14)
        df['adx_14'] = adx['ADX_14']
        ar = ta.aroon(df[self.high_col], df[self.low_col], length=14)
        df['aroon_up'], df['aroon_dn'] = ar['AROONU_14'], ar['AROOND_14']
        df['psar'] = ta.psar(df[self.high_col], df[self.low_col], df[self.price_col])['PSARl_0.02_0.2']
        return df

    def _add_volume_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        # Skip volume-based indicators if volume_col is not available
        return df

    def _add_onchain_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        # mapping of feature name → generator
        funcs: Dict[str, Callable[[pd.DataFrame], pd.Series]] = {
            'addr_growth':     lambda d: d[self.active_addr_col].pct_change(),
            'tx_per_addr':     lambda d: d[self.tx_count_col] / d[self.active_addr_col],
            'nvt':             lambda d: d[self.market_cap_col] / d[self.tx_volume_col],
            'mvrv':            lambda d: d[self.market_cap_col] / d[self.realized_cap_col],
            'nupl':            lambda d: (d[self.market_cap_col]-d[self.realized_cap_col]) / d[self.market_cap_col],
            'cdd_diff':        lambda d: d[self.cdd_col].diff(),
            'hashrate_chg':    lambda d: d[self.hash_rate_col].pct_change(),
            'diff_chg':        lambda d: d[self.difficulty_col].pct_change(),
            'mempool_backlog': lambda d: d[self.mempool_txs_col],
            'fear_greed':      lambda d: d[self.fear_greed_col],
            'sopr':            lambda d: d[self.sopr_col],
            # required additional on-chain
            'realized_price':  lambda d: d[self.realized_price_col],
            'miner_outflows':  lambda d: d[self.miner_outflows_col],
            'exchange_flows':  lambda d: d[self.exchange_flow_col],
            'avg_dormancy':    lambda d: d[self.dormant_coins_col],
            'hodl_1y_share':   lambda d: d[self.hodl_waves_col],
            'miner_revenue':   lambda d: d[self.miner_revenue_col],
            'whale_balance_ratio': lambda d: d[self.whale_balances_col],
            'active_addresses':    lambda d: d[self.active_addrs_col],
            'mev_value':           lambda d: d[self.mev_col],
            'exchange_reserves':   lambda d: d[self.exchange_reserves_col],
            'tx_fees_usd':         lambda d: d[self.tx_fees_col],
            'avg_tx_fee':          lambda d: d[self.tx_fees_col] / d[self.tx_count_col]
        }
        for name, fn in funcs.items():
            try:
                df[name] = fn(df)
            except KeyError:
                # skip if the required column isn't present
                continue
        return df

    def run(self, raw_df: pd.DataFrame) -> pd.DataFrame:
        self._validate(raw_df)
        df = self._set_index(raw_df)
        df = self._clean(df)
        pipeline = [
            self._add_price_features,
            self._add_moving_averages,
            self._add_volatility,
            self._add_momentum,
            self._add_trend_strength,
            self._add_volume_indicators,
            self._add_onchain_metrics
        ]
        for step in pipeline:
            df = step(df)
        # drop columns with zero variance or missing
        df = df.loc[:, df.nunique() > 1].dropna()
        return df

In [346]:
fe = BTCFeatureEngineer()

df_features = fe.run(df_renamed)

# Preview engineered features
df_features.head()

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_datetime,cq_is_shutdown,...,bb_mid,bb_low,rsi_14,macd,macd_signal,macd_hist,adx_14,aroon_up,aroon_dn,psar
date,,,,,,,,,,,,,,,,,,,,,
2023-04-24 10:00:00,1682330400000,1.25,1682330400,65.98,34.02,1682330400,7690.052,6234.039,2023-04-24 10:00:00,0,...,65285.443571,64527.867455,48.251694,-189.646597,-261.862360,72.215763,24.549002,85.714286,35.714286,64531.666514
2023-04-24 11:00:00,1682334000000,1.17,1682334000,66.36,33.64,1682334000,10185.764,8791.377,2023-04-24 11:00:00,0,...,65281.983571,64528.904263,47.707219,-170.614758,-243.612840,72.998082,23.140161,78.571429,28.571429,64566.100984
2023-04-24 12:00:00,1682337600000,0.85,1682337600,66.43,33.57,1682337600,7930.246,9384.912,2023-04-24 12:00:00,0,...,65286.273571,64525.470722,50.355988,-141.798420,-223.249956,81.451536,21.753655,71.428571,21.428571,64599.846764
2023-04-24 13:00:00,1682341200000,1.09,1682341200,65.10,34.90,1682341200,20900.027,19325.408,2023-04-24 13:00:00,0,...,65301.294643,64521.688194,50.084857,-118.792588,-202.358482,83.565894,20.766159,64.285714,14.285714,64632.917629
2023-04-24 14:00:00,1682344800000,0.87,1682344800,64.81,35.19,1682344800,22104.828,24592.664,2023-04-24 14:00:00,0,...,65312.704643,64516.852456,50.610184,-97.236936,-181.334173,84.097237,19.383852,57.142857,7.142857,64665.327076


# Step 3: Data Cleaning

In [396]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.impute import SimpleImputer

class OutlierRemover:
    """
    Removes outlier rows using IsolationForest with a fixed contamination rate.
    Only numeric columns are considered; non-numeric columns and 'date' are preserved.
    """
    def __init__(self):
        self.detector = IsolationForest(contamination=0.05, random_state=0)

    def remove(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        numeric = df.select_dtypes(include=[np.number]).columns.tolist()
        if 'date' in numeric:
            numeric.remove('date')
        preds = self.detector.fit_predict(df[numeric])
        return df.loc[preds == 1].reset_index(drop=True)

class VarianceCorrelationFilter:
    """
    Drops features with zero variance and features exhibiting high collinearity.
    Zero-variance removal uses VarianceThreshold;
    collinearity threshold is fixed at 0.95.
    Always retains key unified price columns.
    """
    def __init__(self):
        self.var_thresh = VarianceThreshold(threshold=0.0)
        self.corr_threshold = 0.95
        self.preserve = {'unified_close', 'unified_high', 'unified_low'}

    def filter(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        numeric = df.select_dtypes(include=[np.number]).columns.tolist()
        if 'date' in numeric:
            numeric.remove('date')
        if not numeric:
            return df
        _ = self.var_thresh.fit_transform(df[numeric])
        kept = [col for col, var in zip(numeric, self.var_thresh.variances_) if var > 0 or col in self.preserve]
        for col in self.preserve:
            if col in df.columns and col not in kept:
                kept.append(col)
        df = df[kept + [c for c in df.columns if c not in numeric]]
        corr = df[kept].corr().abs()
        mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
        to_drop = [col for col in corr.columns if col not in self.preserve and any(corr[col][mask[:, corr.columns.get_loc(col)]] > self.corr_threshold)]
        return df.drop(columns=to_drop)

class TopKSelector:
    """
    Selects the top K numeric features within each source prefix (cg_, cq_, gn_) based on univariate regression
    score to the target price column. Always retains unified_close, unified_high, unified_low, time/date columns.
    """
    def __init__(self, k: int = 50):
        self.k = k

    def select(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        # Columns always kept
        time_cols = [c for c in df.columns if 'time' in c.lower()]
        keep_cols = set(time_cols + (['date'] if 'date' in df.columns else []))
        for fixed in ('unified_close', 'unified_high', 'unified_low'):
            if fixed in df.columns:
                keep_cols.add(fixed)

        # Detect target
        candidates = [c for c in df.columns if c == 'unified_close']
        if not candidates:
            candidates = [c for c in df.columns if 'close' in c.lower()]
        if not candidates:
            candidates = [c for c in df.columns if 'price' in c.lower() or 'usd' in c.lower()]
        if not candidates:
            raise ValueError("No suitable price target column found for TopKSelector")
        target = candidates[0]

        # Prepare features
        numeric = df.select_dtypes(include=[np.number]).columns.tolist()
        feature_cols = [c for c in numeric if c not in keep_cols and c != target]
        if not feature_cols:
            return df

        selected = []
        for prefix in ('cg_', 'cq_', 'gn_'):
            pref_cols = [c for c in feature_cols if c.startswith(prefix)]
            if not pref_cols:
                continue
            X = df[pref_cols]
            y = df[target]
            k_here = min(self.k, len(pref_cols))
            selector = SelectKBest(score_func=f_regression, k=k_here)
            selector.fit(X, y)
            mask = selector.get_support()
            selected += [col for col, keep in zip(pref_cols, mask) if keep]

        # Assemble final columns: target, selected, keep_cols
        final_cols = [target] + selected + list(keep_cols)
        ordered = [c for c in df.columns if c in final_cols]
        return df[ordered]

class BTCFeatureScaler:
    """
    Scales numeric features robustly using scikit-learn's RobustScaler:
     - Centers on median
     - Scales according to IQR
    Automatically ignores non-numeric columns and any column containing 'time'.
    """
    def __init__(self):
        self.scaler = RobustScaler(quantile_range=(25.0, 75.0), with_centering=True, with_scaling=True)

    def scale(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        skip = {col for col in df.columns if 'time' in col.lower()} | {'date'}
        numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns if col not in skip]
        if not numeric_cols:
            return df
        scaled = self.scaler.fit_transform(df[numeric_cols].values)
        df[numeric_cols] = pd.DataFrame(scaled, columns=numeric_cols, index=df.index)
        return df

class NullDuplicateCleaner:
    """
    Cleans a DataFrame by:
      - Dropping duplicate rows
      - Imputing missing numeric values with median
      - Imputing missing categorical/text values with most frequent
    """
    def __init__(self):
        self.num_imputer = SimpleImputer(strategy='median')
        self.cat_imputer = SimpleImputer(strategy='most_frequent')

    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy().drop_duplicates().reset_index(drop=True)
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        cat_cols = df.select_dtypes(include=['object']).columns.tolist()
        if num_cols:
            df[num_cols] = self.num_imputer.fit_transform(df[num_cols])
        if cat_cols:
            df[cat_cols] = self.cat_imputer.fit_transform(df[cat_cols])
        return df

class JSONColumnRemover:
    """
    Drops any column whose values are strings (object dtype).
    """
    def remove(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        string_cols = df.select_dtypes(include=['object']).columns.tolist()
        return df.drop(columns=string_cols)


In [412]:
str_remover = StringRemover()
cleaner = NullDuplicateCleaner()
outlier = OutlierRemover()
scaler = BTCFeatureScaler()
var_filter = VarianceCorrelationFilter()
selector = TopKSelector(k=30)

In [364]:
df_unified

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_datetime,cq_is_shutdown,...,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_to_exchanges_sum,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,unified_open,date
0,1681603200000,1.36,1681603200,46.54,53.46,1681603200,2401.789,1926.312,2023-04-16 00:00:00,0,...,109.999459,124,363.704050,4284.255447,60.769203,65948.825,65969.575,65792.355,65807.110,2023-04-16 00:00:00
1,1681606800000,0.81,1681606800,46.77,53.23,1681606800,6988.972,7992.783,2023-04-16 01:00:00,0,...,0.000000,106,513.558534,5413.792302,54.242128,65923.520,66028.875,65636.035,65948.820,2023-04-16 01:00:00
2,1681610400000,1.22,1681610400,46.65,53.35,1681610400,2191.615,1829.544,2023-04-16 02:00:00,0,...,0.000000,183,336.099293,4539.405297,19.971438,65952.120,65969.410,65851.335,65923.525,2023-04-16 02:00:00
3,1681614000000,0.92,1681614000,46.65,53.35,1681614000,1580.313,1597.408,2023-04-16 03:00:00,0,...,0.000000,146,297.922476,3536.821361,42.400667,66015.080,66027.755,65934.690,65952.120,2023-04-16 03:00:00
4,1681617600000,1.17,1681617600,46.53,53.47,1681617600,2689.337,2147.124,2023-04-16 04:00:00,0,...,0.000000,147,1777.445787,2768.101386,1331.496590,65987.485,66049.655,65880.440,66015.075,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8776,1713207600000,0.94,1713207600,64.33,35.67,1713207600,17937.459,20161.521,2024-04-15 19:00:00,0,...,63.399560,773,1141.313734,12321.903583,41.244664,65197.440,65755.410,64457.640,65524.590,2024-04-15 19:00:00
8777,1713211200000,0.91,1713211200,66.28,33.72,1713211200,5627.547,6340.568,2024-04-15 20:00:00,0,...,47.809850,1059,7198.444685,10852.825216,5883.336388,65266.155,65596.830,64994.235,65197.440,2024-04-15 20:00:00
8778,1713214800000,0.91,1713214800,66.00,34.00,1713214800,6188.427,6767.410,2024-04-15 21:00:00,0,...,31.000000,2794,2500.953529,27914.576777,939.059764,64769.715,65446.950,64434.060,65266.155,2024-04-15 21:00:00
8779,1713218400000,1.08,1713218400,66.00,34.00,1713218400,5606.434,5604.114,2024-04-15 22:00:00,0,...,101.609800,1427,2618.615743,7329.556039,523.631449,64837.210,64958.485,64387.230,64769.715,2024-04-15 22:00:00


In [400]:
df_no_str = str_remover.remove(df_unified)
df_no_str

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_is_shutdown,cq_netflow_total,...,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_to_exchanges_sum,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,unified_open,date
0,1681603200000,1.36,1681603200,46.54,53.46,1681603200,2401.789,1926.312,0,69.900332,...,109.999459,124,363.704050,4284.255447,60.769203,65948.825,65969.575,65792.355,65807.110,2023-04-16 00:00:00
1,1681606800000,0.81,1681606800,46.77,53.23,1681606800,6988.972,7992.783,0,-126.729352,...,0.000000,106,513.558534,5413.792302,54.242128,65923.520,66028.875,65636.035,65948.820,2023-04-16 01:00:00
2,1681610400000,1.22,1681610400,46.65,53.35,1681610400,2191.615,1829.544,0,33.639173,...,0.000000,183,336.099293,4539.405297,19.971438,65952.120,65969.410,65851.335,65923.525,2023-04-16 02:00:00
3,1681614000000,0.92,1681614000,46.65,53.35,1681614000,1580.313,1597.408,0,-229.209968,...,0.000000,146,297.922476,3536.821361,42.400667,66015.080,66027.755,65934.690,65952.120,2023-04-16 03:00:00
4,1681617600000,1.17,1681617600,46.53,53.47,1681617600,2689.337,2147.124,0,200.014597,...,0.000000,147,1777.445787,2768.101386,1331.496590,65987.485,66049.655,65880.440,66015.075,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8776,1713207600000,0.94,1713207600,64.33,35.67,1713207600,17937.459,20161.521,0,-163.542264,...,63.399560,773,1141.313734,12321.903583,41.244664,65197.440,65755.410,64457.640,65524.590,2024-04-15 19:00:00
8777,1713211200000,0.91,1713211200,66.28,33.72,1713211200,5627.547,6340.568,0,-272.749835,...,47.809850,1059,7198.444685,10852.825216,5883.336388,65266.155,65596.830,64994.235,65197.440,2024-04-15 20:00:00
8778,1713214800000,0.91,1713214800,66.00,34.00,1713214800,6188.427,6767.410,0,148.048477,...,31.000000,2794,2500.953529,27914.576777,939.059764,64769.715,65446.950,64434.060,65266.155,2024-04-15 21:00:00
8779,1713218400000,1.08,1713218400,66.00,34.00,1713218400,5606.434,5604.114,0,-370.942593,...,101.609800,1427,2618.615743,7329.556039,523.631449,64837.210,64958.485,64387.230,64769.715,2024-04-15 22:00:00


In [402]:
df_cleaned = cleaner.clean(df_no_str)
df_cleaned

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_is_shutdown,cq_netflow_total,...,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_to_exchanges_sum,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,unified_open,date
0,1.681603e+12,1.36,1.681603e+09,46.54,53.46,1.681603e+09,2401.789,1926.312,0.0,69.900332,...,109.999459,124.0,363.704050,4284.255447,60.769203,65948.825,65969.575,65792.355,65807.110,2023-04-16 00:00:00
1,1.681607e+12,0.81,1.681607e+09,46.77,53.23,1.681607e+09,6988.972,7992.783,0.0,-126.729352,...,0.000000,106.0,513.558534,5413.792302,54.242128,65923.520,66028.875,65636.035,65948.820,2023-04-16 01:00:00
2,1.681610e+12,1.22,1.681610e+09,46.65,53.35,1.681610e+09,2191.615,1829.544,0.0,33.639173,...,0.000000,183.0,336.099293,4539.405297,19.971438,65952.120,65969.410,65851.335,65923.525,2023-04-16 02:00:00
3,1.681614e+12,0.92,1.681614e+09,46.65,53.35,1.681614e+09,1580.313,1597.408,0.0,-229.209968,...,0.000000,146.0,297.922476,3536.821361,42.400667,66015.080,66027.755,65934.690,65952.120,2023-04-16 03:00:00
4,1.681618e+12,1.17,1.681618e+09,46.53,53.47,1.681618e+09,2689.337,2147.124,0.0,200.014597,...,0.000000,147.0,1777.445787,2768.101386,1331.496590,65987.485,66049.655,65880.440,66015.075,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8776,1.713208e+12,0.94,1.713208e+09,64.33,35.67,1.713208e+09,17937.459,20161.521,0.0,-163.542264,...,63.399560,773.0,1141.313734,12321.903583,41.244664,65197.440,65755.410,64457.640,65524.590,2024-04-15 19:00:00
8777,1.713211e+12,0.91,1.713211e+09,66.28,33.72,1.713211e+09,5627.547,6340.568,0.0,-272.749835,...,47.809850,1059.0,7198.444685,10852.825216,5883.336388,65266.155,65596.830,64994.235,65197.440,2024-04-15 20:00:00
8778,1.713215e+12,0.91,1.713215e+09,66.00,34.00,1.713215e+09,6188.427,6767.410,0.0,148.048477,...,31.000000,2794.0,2500.953529,27914.576777,939.059764,64769.715,65446.950,64434.060,65266.155,2024-04-15 21:00:00
8779,1.713218e+12,1.08,1.713218e+09,66.00,34.00,1.713218e+09,5606.434,5604.114,0.0,-370.942593,...,101.609800,1427.0,2618.615743,7329.556039,523.631449,64837.210,64958.485,64387.230,64769.715,2024-04-15 22:00:00


In [404]:
df_no_outliers = outlier.remove(df_cleaned)
df_no_outliers

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_is_shutdown,cq_netflow_total,...,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_to_exchanges_sum,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,unified_open,date
0,1.681603e+12,1.36,1.681603e+09,46.54,53.46,1.681603e+09,2401.789,1926.312,0.0,69.900332,...,109.999459,124.0,363.704050,4284.255447,60.769203,65948.825,65969.575,65792.355,65807.110,2023-04-16 00:00:00
1,1.681607e+12,0.81,1.681607e+09,46.77,53.23,1.681607e+09,6988.972,7992.783,0.0,-126.729352,...,0.000000,106.0,513.558534,5413.792302,54.242128,65923.520,66028.875,65636.035,65948.820,2023-04-16 01:00:00
2,1.681610e+12,1.22,1.681610e+09,46.65,53.35,1.681610e+09,2191.615,1829.544,0.0,33.639173,...,0.000000,183.0,336.099293,4539.405297,19.971438,65952.120,65969.410,65851.335,65923.525,2023-04-16 02:00:00
3,1.681614e+12,0.92,1.681614e+09,46.65,53.35,1.681614e+09,1580.313,1597.408,0.0,-229.209968,...,0.000000,146.0,297.922476,3536.821361,42.400667,66015.080,66027.755,65934.690,65952.120,2023-04-16 03:00:00
4,1.681618e+12,1.17,1.681618e+09,46.53,53.47,1.681618e+09,2689.337,2147.124,0.0,200.014597,...,0.000000,147.0,1777.445787,2768.101386,1331.496590,65987.485,66049.655,65880.440,66015.075,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337,1.713175e+12,0.85,1.713175e+09,61.18,38.82,1.713175e+09,2976.446,3362.971,0.0,771.132911,...,57.820000,829.0,2225.351975,36945.751217,735.115371,67361.895,67552.915,67245.770,67519.020,2024-04-15 10:00:00
8338,1.713179e+12,0.76,1.713179e+09,61.15,38.85,1.713179e+09,5720.496,7818.781,0.0,-367.784666,...,0.000000,592.0,882.376553,6276.670331,65.230959,67044.625,67619.030,67003.720,67361.900,2024-04-15 11:00:00
8339,1.713182e+12,1.08,1.713182e+09,60.80,39.20,1.713182e+09,5468.702,5237.533,0.0,-42.076270,...,0.224769,265.0,307.154358,3717.766705,90.170658,67527.725,67635.700,66976.975,67044.625,2024-04-15 12:00:00
8340,1.713200e+12,0.95,1.713200e+09,63.89,36.11,1.713200e+09,15319.190,15575.760,0.0,-125.540528,...,14.847172,804.0,870.490460,8556.678822,65.716945,65634.590,66608.550,65281.025,66304.860,2024-04-15 17:00:00


In [406]:
df_scaled = scaler.scale(df_no_outliers)
df_scaled

,start_time,cg_longShortRatio,cg_time,cg_longAccount,cg_shortAccount,cg_t,cg_buy,cg_sell,cq_is_shutdown,cq_netflow_total,...,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_to_exchanges_sum,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,unified_open,date
0,1.681603e+12,1.370370,1.681603e+09,-0.772040,0.772040,-1.011853,-0.375654,-0.483806,0.0,0.258809,...,1.516321,-0.367257,-0.905961,-0.300630,-0.388460,0.167992,0.147723,0.177154,0.155015,2023-04-16 00:00:00
1,1.681607e+12,-0.666667,1.681607e+09,-0.756042,0.756042,-1.011616,0.521808,0.712962,0.0,-0.134542,...,-0.087611,-0.446903,-0.761315,-0.049998,-0.411427,0.165489,0.153558,0.161633,0.169040,2023-04-16 01:00:00
2,1.681610e+12,0.851852,1.681610e+09,-0.764389,0.764389,-1.011378,-0.416774,-0.502896,0.0,0.186270,...,-0.087611,-0.106195,-0.932606,-0.244015,-0.532016,0.168318,0.147707,0.183010,0.166537,2023-04-16 02:00:00
3,1.681614e+12,-0.259259,1.681614e+09,-0.764389,0.764389,-1.011141,-0.536372,-0.548691,0.0,-0.339551,...,-0.087611,-0.269912,-0.969456,-0.466478,-0.453094,0.174547,0.153447,0.191286,0.169367,2023-04-16 03:00:00
4,1.681618e+12,0.666667,1.681618e+09,-0.772735,0.772735,-1.010904,-0.319397,-0.440245,0.0,0.519098,...,-0.087611,-0.265487,0.458641,-0.637049,4.082902,0.171817,0.155602,0.185900,0.175598,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337,1.713175e+12,-0.518519,1.713175e+09,0.246218,-0.246218,1.067085,-0.263225,-0.200388,0.0,1.661601,...,0.755478,2.752212,0.890979,6.946608,1.984391,0.307780,0.303511,0.321463,0.324446,2024-04-15 10:00:00
8338,1.713179e+12,-0.851852,1.713179e+09,0.244131,-0.244131,1.067323,0.273636,0.678636,0.0,-0.616765,...,-0.087611,1.703540,-0.405317,0.141465,-0.372760,0.276394,0.310016,0.297430,0.308896,2024-04-15 11:00:00
8339,1.713182e+12,0.333333,1.713182e+09,0.219788,-0.219788,1.067560,0.224374,0.169418,0.0,0.034804,...,-0.084333,0.256637,-0.960545,-0.426328,-0.285003,0.324185,0.311657,0.294774,0.277494,2024-04-15 12:00:00
8340,1.713200e+12,-0.148148,1.713200e+09,0.434707,-0.434707,1.068745,2.151579,2.208902,0.0,-0.132164,...,0.128880,2.641593,-0.416790,0.647375,-0.371050,0.136907,0.210593,0.126384,0.204278,2024-04-15 17:00:00


In [408]:
df_filtered = var_filter.filter(df_scaled)
df_filtered

,start_time,cg_longShortRatio,cg_longAccount,cg_buy,cq_is_shutdown,cq_netflow_total,cq_transactions_count_inflow,cq_transactions_count_outflow,cq_reserve,cq_reserve_usd,...,gn_transactions_transfers_volume_to_exchanges_mean,gn_transactions_transfers_volume_sth_to_exchanges_sum,gn_transactions_transfers_volume_to_otc_desks_sum,gn_transactions_transfers_whales_to_exchanges_count,gn_transactions_transfers_volume_within_exchanges_sum,gn_transactions_transfers_volume_whales_to_exchanges_sum,unified_close,unified_high,unified_low,date
0,1.681603e+12,1.370370,-0.772040,-0.375654,0.0,0.258809,-0.205667,0.863636,0.586101,-0.008144,...,-0.923518,-0.937681,1.516321,-0.367257,-0.300630,-0.388460,0.167992,0.147723,0.177154,2023-04-16 00:00:00
1,1.681607e+12,-0.666667,-0.756042,0.521808,0.0,-0.134542,-0.403108,-0.363636,0.581715,-0.016572,...,-0.612022,-0.952794,-0.087611,-0.446903,-0.049998,-0.411427,0.165489,0.153558,0.161633,2023-04-16 01:00:00
2,1.681610e+12,0.851852,-0.764389,-0.416774,0.0,0.186270,-0.154479,1.727273,0.582880,-0.010499,...,-1.036674,-1.007516,-0.087611,-0.106195,-0.244015,-0.532016,0.168318,0.147707,0.183010,2023-04-16 02:00:00
3,1.681614e+12,-0.259259,-0.764389,-0.536372,0.0,-0.339551,-0.496344,-0.681818,0.574947,-0.013140,...,-0.814042,-1.165157,-0.087611,-0.269912,-0.466478,-0.453094,0.174547,0.153447,0.191286,2023-04-16 03:00:00
4,1.681618e+12,0.666667,-0.772735,-0.319397,0.0,0.519098,-0.092322,0.954545,0.581869,-0.010245,...,1.064108,0.831243,-0.087611,-0.265487,-0.637049,4.082902,0.171817,0.155602,0.185900,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337,1.713175e+12,-0.518519,0.246218,-0.263225,0.0,1.661601,-0.856490,-0.136364,-0.543469,2.634451,...,0.426394,0.827693,0.755478,2.752212,6.946608,1.984391,0.307780,0.303511,0.321463,2024-04-15 10:00:00
8338,1.713179e+12,-0.851852,0.244131,0.273636,0.0,-0.616765,-0.922303,-0.272727,-0.556197,2.586075,...,-0.583615,-0.508512,-0.087611,1.703540,0.141465,-0.372760,0.276394,0.310016,0.297430,2024-04-15 11:00:00
8339,1.713182e+12,0.333333,0.219788,0.224374,0.0,0.034804,-1.079525,-1.227273,-0.557653,2.598477,...,-0.685052,-0.937406,-0.084333,0.256637,-0.426328,-0.285003,0.324185,0.311657,0.294774,2024-04-15 12:00:00
8340,1.713200e+12,-0.148148,0.434707,2.151579,0.0,-0.132164,-0.922303,-0.681818,-0.538458,2.437803,...,-0.654530,-0.460612,0.128880,2.641593,0.647375,-0.371050,0.136907,0.210593,0.126384,2024-04-15 17:00:00


In [416]:
df_final = selector.select(df_filtered)
df_final

,start_time,cg_longShortRatio,cg_longAccount,cg_buy,cq_is_shutdown,cq_netflow_total,cq_transactions_count_inflow,cq_transactions_count_outflow,cq_reserve,cq_reserve_usd,...,gn_supply_active_6m_12m,gn_supply_active_more_1y_percent,gn_supply_active_more_2y_percent,gn_supply_highly_liquid_sum,gn_supply_illiquid_sum,gn_supply_sth_lth_realized_value_ratio,unified_close,unified_high,unified_low,date
0,1.681603e+12,1.370370,-0.772040,-0.375654,0.0,0.258809,-0.205667,0.863636,0.586101,-0.008144,...,-1.916810,-1.667144,-2.523078,0.737491,-0.543701,-0.822327,0.167992,0.147723,0.177154,2023-04-16 00:00:00
1,1.681607e+12,-0.666667,-0.756042,0.521808,0.0,-0.134542,-0.403108,-0.363636,0.581715,-0.016572,...,-1.916980,-1.667046,-2.520901,0.737696,-0.543591,-0.829597,0.165489,0.153558,0.161633,2023-04-16 01:00:00
2,1.681610e+12,0.851852,-0.764389,-0.416774,0.0,0.186270,-0.154479,1.727273,0.582880,-0.010499,...,-1.916834,-1.667106,-2.520835,0.737738,-0.543537,-0.831052,0.168318,0.147707,0.183010,2023-04-16 02:00:00
3,1.681614e+12,-0.259259,-0.764389,-0.536372,0.0,-0.339551,-0.496344,-0.681818,0.574947,-0.013140,...,-1.916684,-1.666965,-2.520701,0.734506,-0.542782,-0.853959,0.174547,0.153447,0.191286,2023-04-16 03:00:00
4,1.681618e+12,0.666667,-0.772735,-0.319397,0.0,0.519098,-0.092322,0.954545,0.581869,-0.010245,...,-1.916324,-1.670948,-2.520828,0.749271,-0.547616,-0.838919,0.171817,0.155602,0.185900,2023-04-16 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337,1.713175e+12,-0.518519,0.246218,-0.263225,0.0,1.661601,-0.856490,-0.136364,-0.543469,2.634451,...,-2.485478,-2.896262,-1.614350,-0.087184,0.322971,1.943330,0.307780,0.303511,0.321463,2024-04-15 10:00:00
8338,1.713179e+12,-0.851852,0.244131,0.273636,0.0,-0.616765,-0.922303,-0.272727,-0.556197,2.586075,...,-2.485172,-2.896401,-1.614262,-0.085519,0.322732,1.928046,0.276394,0.310016,0.297430,2024-04-15 11:00:00
8339,1.713182e+12,0.333333,0.219788,0.224374,0.0,0.034804,-1.079525,-1.227273,-0.557653,2.598477,...,-2.483893,-2.896438,-1.614307,-0.084780,0.322440,1.874437,0.324185,0.311657,0.294774,2024-04-15 12:00:00
8340,1.713200e+12,-0.148148,0.434707,2.151579,0.0,-0.132164,-0.922303,-0.681818,-0.538458,2.437803,...,-2.462448,-2.912825,-1.626196,-0.076093,0.328210,2.418850,0.136907,0.210593,0.126384,2024-04-15 17:00:00


# Step 4: Remove outliers after scaling

In [420]:
out_path = Path("btc_features_output.csv")

try:
    df_final.to_csv(out_path, index=False)
    print(f"✅ Done — features saved to: {out_path}")
except PermissionError:
    print("❌ File is currently open or locked. Please close it and try again.")

✅ Done — features saved to: btc_features_output.csv
